In [ ]:
# This script reads a CSV file and inserts its data into a MariaDB database
# The CSV file must have a header row with the column names
# The database table must have the same column names as the CSV file
# The database connection details are read from environment variables

# Required Python package: mariadb
# Install with: pip install mariadb

# Required environment variables:
# DB_USER: the database username
# DB_PASSWORD: the database password
# DB_HOST: the database host
# DB_PORT: the database port
# DB_NAME: the database name

# The CSV file must be named cwe_cleaned.csv and be in the same directory as this script
# The CSV file must have the following columns: cwe_id, CWE Name, CWE Description

# The database table must be named vuln and have the following columns: cwe_id, name_vuln, desc_vuln
# The columns must be of type VARCHAR
# The cwe_id column must be the primary key
# The cwe_id column must be unique
# The cwe_id column must not be null
# The name_vuln column must not be null
# The desc_vuln column must not be null

# The script will read the CSV file, connect to the database, and insert the data into the vuln table

# Import the required libraries
import mariadb
import csv
import sys
import os

# Connect to MariaDB
try:
    conn = mariadb.connect(
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
        database=os.getenv("DB_NAME")
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB: {e}")
    sys.exit(1)

cursor = conn.cursor()
insert_query = "INSERT INTO vuln (cwe_id, name_vuln, desc_vuln) VALUES (?, ?, ?)"

# Open the CSV file
with open("cwe_cleaned.csv", "r", encoding="utf-8") as csvfile:
    csv_reader = csv.DictReader(csvfile)
    data_rows = []
    for row in csv_reader:
        # Make sure to cast or clean your data as needed
        cwe_id = row["cwe_id"].strip()  # Already a string
        name_vuln = row["CWE Name"].strip()
        desc_vuln = row["CWE Description"].strip()
        # Append as a tuple
        data_rows.append((cwe_id,name_vuln,desc_vuln))

# Insert all rows at once using executemany
try:
    cursor.executemany(insert_query, data_rows)
    conn.commit()
    print("Data inserted successfully!")
except mariadb.Error as e:
    print(f"Error inserting data: {e}")

conn.close()


Data inserted successfully!


In [2]:
import json
import mariadb
import sys

# Connect to your MariaDB database (update with your credentials)
try:
    conn = mariadb.connect(
        user="root",
        password="Shyvarra581.",
        host="localhost",
        port=3306,
        database="hacking_project_bloc1"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB: {e}")
    sys.exit(1)

cursor = conn.cursor()

# --- 1. Create Tables ---

# Domains Table
create_domains = """
CREATE TABLE IF NOT EXISTS domains (
    domain VARCHAR(255) PRIMARY KEY,
    domain_type VARCHAR(50),
    registrar VARCHAR(255),
    whois TEXT,
    reputation INT,
    tld VARCHAR(10),
    last_dns_records_date INT,
    last_analysis_date INT
);
"""
cursor.execute(create_domains)

# DNS_Records Table
create_dns_records = """
CREATE TABLE IF NOT EXISTS dns_records (
    record_id INT AUTO_INCREMENT PRIMARY KEY,
    domain VARCHAR(255),
    record_type VARCHAR(20),
    ttl INT,
    value VARCHAR(255),
    FOREIGN KEY (domain) REFERENCES domains(domain)
);
"""
cursor.execute(create_dns_records)

# Analysis_Stats Table
create_analysis_stats = """
CREATE TABLE IF NOT EXISTS analysis_stats (
    domain VARCHAR(255) PRIMARY KEY,
    malicious INT,
    suspicious INT,
    undetected INT,
    harmless INT,
    FOREIGN KEY (domain) REFERENCES domains(domain)
);
"""
cursor.execute(create_analysis_stats)

# Analysis_Results Table
create_analysis_results = """
CREATE TABLE IF NOT EXISTS analysis_results (
    result_id INT AUTO_INCREMENT PRIMARY KEY,
    domain VARCHAR(255),
    engine_name VARCHAR(100),
    method VARCHAR(50),
    category VARCHAR(50),
    result VARCHAR(50),
    FOREIGN KEY (domain) REFERENCES domains(domain)
);
"""
cursor.execute(create_analysis_results)

# Popularity_Ranks Table
create_popularity_ranks = """
CREATE TABLE IF NOT EXISTS popularity_ranks (
    rank_id INT AUTO_INCREMENT PRIMARY KEY,
    domain VARCHAR(255),
    source VARCHAR(50),
    rank INT,
    timestamp INT,
    FOREIGN KEY (domain) REFERENCES domains(domain)
);
"""
cursor.execute(create_popularity_ranks)

# Categories Table
create_categories = """
CREATE TABLE IF NOT EXISTS categories (
    cat_id INT AUTO_INCREMENT PRIMARY KEY,
    domain VARCHAR(255),
    source VARCHAR(50),
    category_description VARCHAR(255),
    FOREIGN KEY (domain) REFERENCES domains(domain)
);
"""
cursor.execute(create_categories)

# --- 2. Insert Data from JSON File ---

# Load the JSON file (assumed to be a list of domain reports)
with open('vt_domains_results.json', 'r', encoding='utf-8') as f:
    reports = json.load(f)

for item in reports:
    data = item.get("data", {})
    domain = data.get("id", "")
    domain_type = data.get("type", "")
    
    attributes = data.get("attributes", {})
    registrar = attributes.get("registrar", None)
    whois = attributes.get("whois", None)
    reputation = attributes.get("reputation", None)
    tld = attributes.get("tld", None)
    last_dns_records_date = attributes.get("last_dns_records_date", None)
    last_analysis_date = attributes.get("last_analysis_date", None)
    
    # Insert into Domains table
    try:
        cursor.execute("""
            INSERT INTO domains (domain, domain_type, registrar, whois, reputation, tld, last_dns_records_date, last_analysis_date)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ON DUPLICATE KEY UPDATE
                domain_type = VALUES(domain_type),
                registrar = VALUES(registrar),
                whois = VALUES(whois),
                reputation = VALUES(reputation),
                tld = VALUES(tld),
                last_dns_records_date = VALUES(last_dns_records_date),
                last_analysis_date = VALUES(last_analysis_date);
        """, (domain, domain_type, registrar, whois, reputation, tld, last_dns_records_date, last_analysis_date))
    except mariadb.Error as e:
        print(f"Error inserting domain {domain}: {e}")
    
    # Insert DNS Records
    dns_records = attributes.get("last_dns_records", [])
    for record in dns_records:
        record_type = record.get("type")
        ttl = record.get("ttl")
        value = record.get("value")
        try:
            cursor.execute("""
                INSERT INTO dns_records (domain, record_type, ttl, value)
                VALUES (?, ?, ?, ?);
            """, (domain, record_type, ttl, value))
        except mariadb.Error as e:
            print(f"Error inserting DNS record for {domain}: {e}")
    
    # Insert Analysis Stats
    stats = attributes.get("last_analysis_stats", {})
    malicious = stats.get("malicious", 0)
    suspicious = stats.get("suspicious", 0)
    undetected = stats.get("undetected", 0)
    harmless = stats.get("harmless", 0)
    try:
        cursor.execute("""
            INSERT INTO analysis_stats (domain, malicious, suspicious, undetected, harmless)
            VALUES (?, ?, ?, ?, ?)
            ON DUPLICATE KEY UPDATE
                malicious = VALUES(malicious),
                suspicious = VALUES(suspicious),
                undetected = VALUES(undetected),
                harmless = VALUES(harmless);
        """, (domain, malicious, suspicious, undetected, harmless))
    except mariadb.Error as e:
        print(f"Error inserting analysis stats for {domain}: {e}")
    
    # Insert Analysis Results per engine
    analysis_results = attributes.get("last_analysis_results", {})
    for engine, result_data in analysis_results.items():
        method = result_data.get("method")
        category = result_data.get("category")
        result_val = result_data.get("result")
        try:
            cursor.execute("""
                INSERT INTO analysis_results (domain, engine_name, method, category, result)
                VALUES (?, ?, ?, ?, ?);
            """, (domain, engine, method, category, result_val))
        except mariadb.Error as e:
            print(f"Error inserting analysis result for {domain} engine {engine}: {e}")
    
    # Insert Popularity Ranks
    popularity_ranks = attributes.get("popularity_ranks", {})
    for source, rank_info in popularity_ranks.items():
        rank = rank_info.get("rank")
        ts = rank_info.get("timestamp")
        try:
            cursor.execute("""
                INSERT INTO popularity_ranks (domain, source, rank, timestamp)
                VALUES (?, ?, ?, ?);
            """, (domain, source, rank, ts))
        except mariadb.Error as e:
            print(f"Error inserting popularity rank for {domain} source {source}: {e}")
    
    # Insert Categories (if any)
    categories = attributes.get("categories", {})
    for source, cat_desc in categories.items():
        try:
            cursor.execute("""
                INSERT INTO categories (domain, source, category_description)
                VALUES (?, ?, ?);
            """, (domain, source, cat_desc))
        except mariadb.Error as e:
            print(f"Error inserting category for {domain} source {source}: {e}")

# Commit all changes
try:
    conn.commit()
    print("All data inserted successfully!")
except mariadb.Error as e:
    print(f"Error committing data: {e}")
    conn.rollback()

conn.close()



All data inserted successfully!
